In [49]:
# Rename the files

import os

def rename(x):
    new = x.split('_')
    two = new[1].split('.txt')[0]
    return two + '_' + new[0] + '.txt'

directory = os.listdir("C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\o+p")

for i in directory:
#     print(rename(i))
    os.rename("C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\o+p\\" + i, "C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\o+p\\" + rename(i))



In [7]:
# Pre-processing functions

import re
import string
import time

regex = re.compile('[%s]' % re.escape(string.punctuation))
# test = 'a bc defg hij k && lmno pqr stuv \t wxy z \n abcd nate is the  ;best  '  

def re_remove_punct(s):  # From Vinko's solution, with fix.
    return regex.sub('', s)

def remove_spaces(s, extra = False):
    s = s.lower()
    s = re_remove_punct(s)
    s = " ".join(s.split())
    if extra == True:
        s = s.replace(' ','')
    return s

def create_shingled_features(s, k, clean = False, remove_all_spaces= False, print_to_file = False, res = 'result ' + time.strftime("%Y%m%d-%H%M%S") + '.txt'):
    output = []
    if clean == True:
        s = remove_spaces(s, extra = remove_all_spaces)
    f = open(res, 'a')
    while True:
        result = (s[:k], s[1:])
        s = result[1]
        if result[0] == '':
            break
        if print_to_file == True:
            f.write(result[0] + '\n')
        else:
            output.append(result[0])
    f.close()
    if print_to_file == True:
        print("File printed to '" + res + "'")
    else:
        return output


In [58]:
# Pre-process and Simhash all the files, place into a dictionary called "all_simhasht"

from simhash import Simhash, SimhashIndex
from collections import OrderedDict
### Create hamming distance matrix

#### import Shari's plagiarized samples

import glob
path="C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\o+p\\*.txt"
chop = "C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\o+p\\"

all_dict = {} #store Shari's sample plagiarism files here

for filename in glob.iglob(path):
    chopt = filename.replace(chop,'')
    with open(filename, encoding="ISO-8859-1") as f:
        all_dict[chopt] = f.read()
    
all_dict = OrderedDict(sorted(all_dict.items(), key=lambda t: t[0]))    

keys = list(all_dict.keys())

### Create the index of simhashed files

objs = [(str(k), Simhash(create_shingled_features(s, 3, clean=True))) for k, s in all_dict.items()]
index = SimhashIndex(objs, k=3)

all_simhasht = {}

for i in objs:
    all_simhasht[i[0]] = i[1]
    
all_simhasht = OrderedDict(sorted(all_simhasht.items(), key=lambda t: t[0]))    

In [67]:
# Calculate Hamming Distances for each simhash'd file and input into a matrix

import numpy as np
import pandas as pd

# np.set_printoptions(threshold=np.nan)
distances = np.empty(len(all_simhasht))

for i in all_simhasht:
    m = []
    for j in all_simhasht:
        m.append(all_simhasht[j].distance(all_simhasht[i]))
    distances = np.vstack([distances, m])
    
print(all_simhasht.keys())    
distances = np.delete(distances, (0), axis=0) # for some reason, the first row is all 0s

df = pd.DataFrame(distances)
df.astype(int)


odict_keys(['taska_g0pA.txt', 'taska_g0pB.txt', 'taska_g0pC.txt', 'taska_g0pD.txt', 'taska_g0pE.txt', 'taska_g1pA.txt', 'taska_g1pB.txt', 'taska_g1pD.txt', 'taska_g2pA.txt', 'taska_g2pB.txt', 'taska_g2pC.txt', 'taska_g2pE.txt', 'taska_g3pA.txt', 'taska_g3pB.txt', 'taska_g3pC.txt', 'taska_g4pB.txt', 'taska_g4pC.txt', 'taska_g4pD.txt', 'taska_g4pE.txt', 'taska_orig.txt', 'taskb_g0pA.txt', 'taskb_g0pB.txt', 'taskb_g0pC.txt', 'taskb_g0pD.txt', 'taskb_g0pE.txt', 'taskb_g1pA.txt', 'taskb_g1pB.txt', 'taskb_g1pD.txt', 'taskb_g2pA.txt', 'taskb_g2pB.txt', 'taskb_g2pC.txt', 'taskb_g2pE.txt', 'taskb_g3pA.txt', 'taskb_g3pB.txt', 'taskb_g3pC.txt', 'taskb_g4pB.txt', 'taskb_g4pC.txt', 'taskb_g4pD.txt', 'taskb_g4pE.txt', 'taskb_orig.txt', 'taskc_g0pA.txt', 'taskc_g0pB.txt', 'taskc_g0pC.txt', 'taskc_g0pD.txt', 'taskc_g0pE.txt', 'taskc_g1pA.txt', 'taskc_g1pB.txt', 'taskc_g1pD.txt', 'taskc_g2pA.txt', 'taskc_g2pB.txt', 'taskc_g2pC.txt', 'taskc_g2pE.txt', 'taskc_g3pA.txt', 'taskc_g3pB.txt', 'taskc_g3pC.txt'

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,19,21,24,24,21,20,18,20,14,...,27,28,24,28,29,27,30,29,23,27
1,19,0,24,23,25,16,17,23,19,13,...,24,23,25,27,22,26,27,28,24,26
2,21,24,0,15,13,18,15,7,15,21,...,22,21,19,21,24,22,23,22,22,18
3,24,23,15,0,8,15,14,16,18,20,...,19,24,20,24,29,23,26,21,25,23
4,24,25,13,8,0,17,14,10,20,22,...,17,20,16,20,29,19,20,19,23,19
5,21,16,18,15,17,0,19,19,21,17,...,20,21,23,23,22,20,23,22,24,22
6,20,17,15,14,14,19,0,16,14,18,...,21,24,18,22,25,23,22,21,21,19
7,18,23,7,16,10,19,16,0,16,20,...,21,20,16,22,25,23,24,23,21,19
8,20,19,15,18,20,21,14,16,0,14,...,21,22,16,22,19,27,20,23,19,21
9,14,13,21,20,22,17,18,20,14,0,...,23,24,20,22,23,25,22,25,21,21


In [63]:
# Write to an Excel File

writer = pd.ExcelWriter('plagiarism.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
import re
import string
import time
import numpy as np
import pandas as pd
from simhash import Simhash, SimhashIndex
from collections import OrderedDict
import glob